In [1]:
import torch
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from typing import Tuple
import re
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from tqdm.auto import tqdm

from google.colab import drive



In [2]:
DRIVE_MOUNT_PATH = "/content/drive/MyDrive/journeyhacks 2026 data/"
DATASET_TRAINING_PATH = "training_dataset/"
DATASET_TESTING_PATH = "testing_dataset/"
MODEL_OUTPUT_PATH = "model/"

In [3]:
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
def is_valid_review(text):
    text = text.strip()

    # HTML / XML
    if text.startswith("<"):
        return False
    if re.search(r"<[^>]+>", text):
        return False

    # Stack traces / logs
    if any(keyword in text.lower() for keyword in [
        "exception", "traceback", "error:", "nullpointer"
    ]):
        return False

    return True

In [5]:
def clean_data() -> None:
    df = pd.read_csv(f"{DRIVE_MOUNT_PATH}fake_reviews_dataset.csv")

    print(df.shape)
    df = df[df["label"] == 0][["rating","text"]]
    print(df.shape)

    # Check missing values
    print(df.isna().any(axis=1).sum())

    # Remove html tag rows
    df = df[df["text"].apply(is_valid_review)]
    print(df.shape)

    # Print maximum words
    #word_counts = df["text"].str.split().str.len()
    #buckets = word_counts.value_counts().sort_index()
    #print(buckets)

    df.to_csv(f"{DRIVE_MOUNT_PATH}fake_reviews_dataset_cleaned.csv",index=False)

In [6]:
def normalize_data() -> None:
    label_col = "rating"
    random_state = 42

    df = pd.read_csv(f"{DRIVE_MOUNT_PATH}fake_reviews_dataset_cleaned.csv")
    label_counts = df[label_col].value_counts()
    min_count = label_counts.min()

    # sample each label group down to min_count
    balanced_df = (
        df.groupby(label_col, group_keys=False)
          .apply(lambda x: x.sample(n=min_count, random_state=random_state))
          .reset_index(drop=True)
    )

    balanced_df.to_csv(f"{DRIVE_MOUNT_PATH}fake_reviews_dataset_normalized.csv",index=False)

In [7]:
def split_data() -> None:
    df = pd.read_csv(f"{DRIVE_MOUNT_PATH}fake_reviews_dataset_normalized.csv")

    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

    test_df.to_csv(f"{DRIVE_MOUNT_PATH}{DATASET_TESTING_PATH}test.csv",index=False)

    train_df, validation_df = train_test_split(df, test_size=0.2, random_state=42)

    train_df.to_csv(f"{DRIVE_MOUNT_PATH}{DATASET_TRAINING_PATH}train.csv",index=False)
    validation_df.to_csv(f"{DRIVE_MOUNT_PATH}{DATASET_TRAINING_PATH}validation.csv",index=False)




In [8]:
def read_data() -> Tuple[list,list,list]:
    train_df = pd.read_csv(f"{DRIVE_MOUNT_PATH}{DATASET_TRAINING_PATH}train.csv")
    validation_df = pd.read_csv(f"{DRIVE_MOUNT_PATH}{DATASET_TRAINING_PATH}validation.csv")
    test_df = pd.read_csv(f"{DRIVE_MOUNT_PATH}{DATASET_TESTING_PATH}test.csv")

    train_pairs = list(zip(train_df["text"].tolist(), train_df["rating"].tolist()))
    validation_pairs = list(zip(validation_df["text"].tolist(), validation_df["rating"].tolist()))
    test_pairs = list(zip(test_df["text"].tolist(), test_df["rating"].tolist()))


    return train_pairs, validation_pairs, test_pairs


In [9]:
class ReviewsDataset(Dataset):
    def __init__(self, pairs, tokenizer, max_len=128):
        self.labels = []
        texts = []
        for text, label in pairs:
            if 1 <= label <= 5:
                label -= 1
            texts.append(text)
            self.labels.append(label)

        self.enc = tokenizer(
            texts,
            truncation=True,
            padding="max_length",
            max_length=max_len,
        )

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.enc["input_ids"][idx], dtype=torch.long),
            "attention_mask": torch.tensor(self.enc["attention_mask"][idx], dtype=torch.long),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }


# ----------------------------
# 2) Dataloaders
# ----------------------------
def make_loaders(train_pairs, validation_pairs, model_name="distilbert-base-uncased", batch_size=16, max_len=128):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    train_ds = ReviewsDataset(train_pairs, tokenizer, max_len=max_len)
    validation_ds = ReviewsDataset(validation_pairs, tokenizer, max_len=max_len)

    train_loader = DataLoader(train_ds, batch_size=batch_size,num_workers=4,pin_memory=True, persistent_workers=True,shuffle=True)
    validation_loader = DataLoader(validation_ds, batch_size=batch_size,num_workers=4,pin_memory=True, persistent_workers=True,shuffle=False)

    return tokenizer, train_loader, validation_loader


# ----------------------------
# 3) Train + Eval
# ----------------------------
@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    total, correct = 0, 0
    total_loss = 0.0

    for batch in loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        out = model(**batch)
        loss = out.loss
        logits = out.logits

        preds = logits.argmax(dim=-1)
        correct += (preds == batch["labels"]).sum().item()
        total += batch["labels"].size(0)
        total_loss += loss.item() * batch["labels"].size(0)

    return total_loss / total, correct / total


def train_model(train_pairs, validation_pairs, model_name="distilbert-base-uncased",
                lr=5e-5, epochs=10, batch_size=16, max_len=128):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    tokenizer, train_loader, validation_loader = make_loaders(
        train_pairs, validation_pairs, model_name=model_name, batch_size=batch_size, max_len=max_len
    )

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=lr)

    # scheduler (optional but standard)
    steps_per_epoch = len(train_loader)
    total_steps = steps_per_epoch * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        seen = 0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=True)
        for batch in pbar:
            batch = {k: v.to(device) for k, v in batch.items()}

            optimizer.zero_grad(set_to_none=True)
            out = model(**batch)
            loss = out.loss
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()

            bs = batch["labels"].size(0)
            running_loss += loss.item() * bs
            seen += bs

            pbar.set_postfix(loss=f"{loss.item():.4f}", avg=f"{(running_loss/seen):.4f}")

        train_loss = running_loss / seen
        validation_loss, validation_acc = evaluate(model, validation_loader, device)

        print(f"Epoch {epoch}/{epochs} | train_loss={train_loss:.4f} | validation_loss={validation_loss:.4f} | validation_acc={validation_acc:.4f}")

    return model, tokenizer

In [10]:
def test_model(model, tokenizer, test_pairs) -> None:
   test_ds = ReviewsDataset(test_pairs, tokenizer, max_len=128)
   test_loader = DataLoader(
      test_ds,
      batch_size=16,
      shuffle=False,
      num_workers=4,
      pin_memory=True,
      persistent_workers=True,
   )
   device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   model.to(device)
   test_loss, test_acc = evaluate(model, test_loader, device)

   print(f"Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")


In [11]:
def save_model(model, tokenizer) -> None:
    model.save_pretrained(f"{DRIVE_MOUNT_PATH}{MODEL_OUTPUT_PATH}model")
    tokenizer.save_pretrained(f"{DRIVE_MOUNT_PATH}{MODEL_OUTPUT_PATH}tokenizer")

In [12]:
clean_data()
normalize_data()
split_data()

(40526, 4)
(20232, 2)
0
(19970, 2)


/tmp/ipython-input-314964860.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min_count, random_state=random_state))


In [13]:
train_pairs, validation_pairs,test_pairs =read_data()

In [14]:
model, tokenizer = train_model(train_pairs, validation_pairs)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be us

Epoch 1/10:   0%|          | 0/248 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
test_model(model, tokenizer, test_pairs)

In [ ]:
save_model(model, tokenizer)